In [1]:
import pandas as pd
import time
import re
import os
import json
from tqdm import tqdm
from copy import deepcopy

import openai
from dotenv import load_dotenv

In [21]:
file_path = "data/med_expert.csv"
df = pd.read_csv(file_path, encoding='utf-8')
pd.options.mode.chained_assignment = None

In [ ]:
def prepare_qna_data(df):
    def extract_text_data(df):
        extracted_df = df[(df['question_photo'] == "['사진 없음']") &
                        (df['answer_photo'] == "['사진 없음']") &
                        (df['question_video'] == '동영상 없음') &
                        (df['answer_video'] == '동영상 없음')]
        return extracted_df
    
    initial_row_count = len(df)
    df = extract_text_data(df)
    extracted_row_count = len(df)
    # df = df.sample(n=10, random_state=42)  # 샘플링
    df['id'] = list(range(len(df)))
    print(f"원본 데이터 크기: {initial_row_count}")
    print(f"추출된 데이터 크기: {extracted_row_count}")
    # print(f"샘플링된 데이터 크기: {len(df)}")
    
    return df[['id', '제목', '본문', '답변', 'answer_date']]

df = prepare_qna_data(df)
df.info()

원본 데이터 크기: 25930
추출된 데이터 크기: 15798
<class 'pandas.core.frame.DataFrame'>
Index: 15798 entries, 0 to 25929
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15798 non-null  int64 
 1   제목           15798 non-null  object
 2   본문           14928 non-null  object
 3   답변           15798 non-null  object
 4   answer_date  15798 non-null  object
dtypes: int64(1), object(4)
memory usage: 740.5+ KB


In [6]:
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    
    if hours > 0:
        return f"처리 시간: {hours}시간 {minutes}분 {seconds}초"
    elif minutes > 0:
        return f"처리 시간: {minutes}분 {seconds}초"
    else:
        return f"처리 시간: {seconds}초"

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Step 1: 필터링

In [8]:
with open("prompt/filtering_system.txt", "r", encoding="utf-8") as file:
    system_prompt = file.read()
    
with open("prompt/filtering_user.txt", "r", encoding="utf-8") as file:
    base_user_prompt = file.read()

In [ ]:
# def filter_text(title, text, answer):
#     user_prompt = base_user_prompt.replace("{title}", title).replace("{text}", str(text)).replace("{answer}", answer)
    
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": user_prompt}],
#         seed=42
#     )
    
#     return response.choices[0].message.content.strip()

# filtering_outputs = []

# start_time = time.time()
# for _, row in df.iterrows():
#     filtering_outputs.append(filter_text(row['제목'], row['본문'], row['답변']))
    
# df['is_relevant'] = filtering_outputs

# output_file = "data/filtered_data.json"
# df.to_json(output_file, orient='records', force_ascii=False, indent=4)
# end_time = time.time()
# elapsed_time = end_time - start_time

# print(f"필터링 결과 저장: {output_file}")
# print(f"전체 처리 시간: {elapsed_time:.2f}초")

In [19]:
output_file = "data/filtered_data.json"

if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        filtered_data = json.load(f)
    start_idx = len(filtered_data)
    print(f"{start_idx}개까지 처리됨. 이어서 시작")
else:
    filtered_data = []
    start_idx = 0
    print("새로 시작")

def filter_text(title, text, answer):
    user_prompt = base_user_prompt.replace("{title}", title).replace("{text}", str(text)).replace("{answer}", answer)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}],
        seed=42
    )
    
    return response.choices[0].message.content.strip()

df_len = len(df)

for i, row in tqdm(df.iloc[start_idx:].iterrows(), total=df_len - start_idx, desc="Filtering"):
    output = filter_text(row['제목'], row['본문'], row['답변'])
    # time.sleep(3)
    df.loc[i, 'is_relevant'] = output
    
    filtered_data.append(df.loc[i].to_dict())
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print(f"필터링 결과 저장 완료: {output_file}")

새로 시작


Filtering:   0%|          | 40/15798 [00:24<2:43:15,  1.61it/s]


KeyboardInterrupt: 

In [ ]:
def filter_text(title, text, answer):
    user_prompt = base_user_prompt.replace("{title}", title).replace("{text}", str(text)).replace("{answer}", answer)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}],
        seed=42
    )
    
    return response.choices[0].message.content.strip()

output_file = "data/filtered_data.json"
df_len = len(df)

for i, row in tqdm(df.iterrows(), total=df_len, desc="Filtering"):
    output = filter_text(row['제목'], row['본문'], row['답변'])
    df.loc[i, 'is_relevant'] = output
    df.iloc[:i+1].to_json(output_file, orient='records', force_ascii=False, indent=4)
    
print(f"필터링 결과 저장: {output_file}")

#### Batch API

In [ ]:
init_template = {
    "custom_id": None,
    "method": "POST", 
    "url": "/v1/chat/completions",
    "body": {"model": "gpt-4o-mini", 
             "messages":[
                 {"role": "system", "content": system_prompt},
                 ],
             "max_tokens": 1000
             }
    }

In [ ]:
batches = []
def make_batches(title, text, answer, i):
    user_prompt = base_user_prompt.replace("{title}", title).replace("{text}", str(text)).replace("{answer}", answer)
    temp = deepcopy(init_template)
    temp['custom_id'] = f'{i}'
    temp['body']['messages'].append({"role": "user", "content": user_prompt})
    batches.append(temp)

In [ ]:
for _, row in df.iterrows():
    make_batches(row['제목'], row['본문'], row['답변'], row['id'])

os.makedirs("data", exist_ok=True)
file_path = "data/batch_input_filtering.jsonl"
with open(file_path, 'w') as file:
    for item in batches:
        json_string = json.dumps(item, ensure_ascii=False)
        file.write(json_string + '\n')

In [ ]:
batch_input_file = client.files.create(
    file=open(file_path, "rb"),
    purpose="batch"
)

print(batch_input_file)

In [ ]:
batch_input_file_id = batch_input_file.id
batch_job = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "preprocessing"
    }
)

In [ ]:
batch = client.batches.retrieve(batch_job.id)

In [ ]:
MAX_WAIT_TIME = 24 * 60 * 60  
start_time = time.time()


try:
    while True:
        if batch.status == "completed":
            print("배치 작업이 성공적으로 완료되었습니다.")
            break
        
        if time.time() - start_time > MAX_WAIT_TIME:
            print(f"최대 대기 시간({MAX_WAIT_TIME/60}분)을 초과했습니다.")
            break
        
        time.sleep(5)
    
    output_file_id = batch.output_file_id
    file_response = client.files.content(output_file_id).content
    
    result_file_name = "data/batch_output_filtering.jsonl"
    with open(result_file_name, "wb") as f:
        f.write(file_response)
    
    print(f"필터링 완료: {result_file_name}")
    print(f"총 소요 시간: {(time.time() - start_time)/3600:.2f}시간")
    
except Exception as e:
    print(f"배치 작업 처리 중 오류 발생: {e}")
    # 로깅 등 추가 오류 처리 로직

In [ ]:
filtering_outputs = []
with open(result_file_name, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        content = data["response"]["body"]["choices"][0]["message"]["content"]
        filtering_outputs.append(content)

df['is_relevant'] = filtering_outputs
output_file = "data/filtered_data.json"
df.to_json(output_file, orient='records', force_ascii=False, indent=4)
print(f"필터링 결과 저장: {output_file}")

### NaN 제거

In [ ]:
df['is_relevant'].value_counts()

In [ ]:
initial_row_count = len(df)
df = df[df['is_relevant'] == 'True']
dropped_row_count = initial_row_count - len(df)
print(f"드롭된 행의 개수: {dropped_row_count}개")

중복되는 질문 제거

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch

def analyze_duplicate_questions(input_file, output_file, similarity_threshold=0.8):
    """
    유사한 질문들을 분석하고 시각화하는 함수
    
    Parameters:
    - input_file (str): 입력 CSV 파일 경로
    - output_file (str): 분석 결과 CSV 파일 경로
    - similarity_threshold (float): 유사도 임계값 (기본값: 0.8)
    
    Returns:
    - pandas.DataFrame: 유사한 질문 쌍 정보 데이터프레임
    """
    # 데이터 로드
    df = pd.read_csv(input_file)
    
    # Sentence BERT 모델 로드
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')
    
    # 질문들에 대한 임베딩 생성
    question_embeddings = model.encode(df['question'].tolist(), convert_to_tensor=True)
    
    # 코사인 유사도 계산
    similarity_matrix = cosine_similarity(
        question_embeddings.cpu().numpy(), 
        question_embeddings.cpu().numpy()
    )
    
    # 유사한 질문 쌍 저장할 리스트
    similar_questions = []
    
    # 유사한 질문 쌍 찾기
    for i in range(len(df)):
        similar_indices = np.where(
            (similarity_matrix[i] > similarity_threshold) &  # 엄격하게 초과로 변경
            (similarity_matrix[i] < 1.0 - 1e-10)  # 완전히 동일한 행 제외
        )[0]
        
        for idx in similar_indices:
            if df.loc[i, 'question'] != df.loc[idx, 'question']:  # 텍스트 비교 추가
                similar_questions.append({
                    'original_question': df.loc[i, 'question'],
                    'similar_question': df.loc[idx, 'question'],
                    'similarity_score': float('{:.2f}'.format(similarity_matrix[i, idx]))
                })
    
    # 유사한 질문 쌍 데이터프레임 생성
    similar_questions_df = pd.DataFrame(similar_questions)
    
    # 중복 제거 (동일한 질문 쌍 제거)
    similar_questions_df = similar_questions_df.drop_duplicates(
        subset=['original_question', 'similar_question']
    )
    
    # 결과 파일 저장
    # similar_questions_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    similar_questions_df.to_json(output_file.replace('.csv', '.json'), 
                                  orient='records', 
                                  force_ascii=False, 
                                  indent=4)
    
    # 유사한 질문 쌍 출력
    print("유사한 질문 쌍")
    print("------------------")
    for _, row in similar_questions_df.iterrows():
        print(f"\n원본 질문: {row['original_question']}")
        print(f"유사한 질문: {row['similar_question']}")
        print(f"유사도 점수: {row['similarity_score']:.2f}")
    
    # 통계 출력
    print(f"\n총 유사한 질문 쌍 수: {len(similar_questions_df)}")
    
    return similar_questions_df

# 사용 예시
input_path = "data/processed_data.csv"
output_path = "data/similar_questions.csv"

# 유사한 질문 분석
analyze_duplicate_questions(
    input_file=input_path, 
    output_file=output_path,
    # similarity_threshold=0.9
)

# Step 2: 클리닝

In [ ]:
def remove_common_greetings(text):
    if not isinstance(text, str):
        return ''
    
    # 유니코드 특수문자(제로폭 공백 등) 제거
    text = re.sub(r'[\u200b\u200c\u200d\u2060\ufeff]', '', text)
    
    patterns = [
        # 시작 문구
        r'^안녕하세요.*?입니다\.?\s*',
        
        # 끝 문구
        r'\s*감사합니다\.?\s*$',
        r'\s*고맙습니다\.?\s*$',
        r'\s*안녕히\s*계세요\.?\s*$',
        r'\s*안녕히\s*가세요\.?\s*$',
        r'\s*수고하세요\.?\s*$',
    ]
    
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE | re.UNICODE).strip()
    
    return text

df['답변'] = df['답변'].apply(remove_common_greetings)

In [ ]:
with open("prompt/cleaning_system.txt", "r", encoding="utf-8") as file:
    system_prompt = file.read()
    
with open("prompt/cleaning_user.txt", "r", encoding="utf-8") as file:
    base_user_prompt = file.read()

#### Batch API

In [ ]:
batches = []
def make_batches(title, text, answer, i):
    user_prompt = base_user_prompt.replace("{title}", title).replace("{text}", str(text)).replace("{answer}", answer)
    temp = deepcopy(init_template)
    temp['custom_id'] = f'{i}'
    temp['body']['messages'].append({"role": "user", "content": user_prompt})
    batches.append(temp)

In [ ]:
for _, row in df.iterrows():
    make_batches(row['제목'], row['본문'], row['답변'], row['id'])
    
os.makedirs("data", exist_ok=True)
file_path = "data/batch_input_cleaning.jsonl"
with open(file_path, 'w') as file:
    for item in batches:
        json_string = json.dumps(item, ensure_ascii=False)
        file.write(json_string + '\n')

In [ ]:
batch_input_file = client.files.create(
    file=open(file_path, "rb"),
    purpose="batch"
)

print(batch_input_file)

In [ ]:
batch_input_file_id = batch_input_file.id
batch_job = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "preprocessing"
    }
)

In [ ]:
MAX_WAIT_TIME = 24 * 60 * 60  
start_time = time.time()

try:
    while True:
        batch = client.batches.retrieve(batch_job.id)
        
        if batch.status == "completed":
            print("배치 작업이 성공적으로 완료되었습니다.")
            break
        
        if time.time() - start_time > MAX_WAIT_TIME:
            print(f"최대 대기 시간: {MAX_WAIT_TIME//3600}시간을 초과했습니다.")
            break
        
        if batch.status == "failed":
            print("배치 작업이 실패했습니다.")
            print(batch.incomplete_details)
            break
        
        # 일부 실패 시 (status는 completed지만 error_file_id가 존재)
        if batch.error_file_id:
            print("일부 배치 작업이 실패했습니다.")
            error_file = client.files.content(batch.error_file_id)
            with open("error_file.jsonl", "wb") as f:
                f.write(error_file)
            print(error_file)
        
        time.sleep(5)
    
    output_file_id = batch.output_file_id
    file_response = client.files.content(output_file_id).content
    
    result_file_name = "data/batch_output_filtering.jsonl"
    with open(result_file_name, "wb") as f:
        f.write(file_response)
    
    elapsed = time.time() - start_time
    print(f"필터링 완료: {result_file_name}")
    print(f"총 소요 시간: {elapsed//3600}시간 {(elapsed%3600)//60}분")
    
except Exception as e:
    print(f"배치 작업 처리 중 오류 발생: {e}")

In [ ]:
print(batch.status)

In [ ]:
cleaned_questions = []
cleaned_answers = []
with open(result_file_name, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        content = data["response"]["body"]["choices"][0]["message"]["content"]
        
        question_match = re.search(r"클리닝된 질문:\s*(.+)", content)
        answer_match = re.search(r"클리닝된 답변:\s*(.+)", content)
        
        cleaned_question = question_match.group(1).strip() if question_match else ""
        cleaned_question = "" if pd.isna(cleaned_question) else cleaned_question
        cleaned_answer = answer_match.group(1).strip() if answer_match else ""
        cleaned_answer = "" if pd.isna(cleaned_answer) else cleaned_answer
        
        cleaned_questions.append(cleaned_question)
        cleaned_answers.append(cleaned_answer)

df['cleaned_question'] = cleaned_questions
df['cleaned_answer'] = cleaned_answers

output_file = "data/cleaned_data.json"
df.to_json(output_file, orient='records', force_ascii=False, indent=4)
print(f"클리닝 결과 저장: {output_file}")

In [ ]:
df.info()

In [ ]:
null_df = df[((df['question'] == "") | (df['answer'] == ""))]
null_df

In [ ]:
df = df[~((df['question'] == "") | (df['answer'] == ""))]
df['question'] = df['cleaned_question']
df['answer'] = df['cleaned_answer']
df = df[['id', 'question', 'answer', 'answer_date']]

output_file = "data/preprocessed_data.json"
df.to_json(output_file, orient='records', force_ascii=False, indent=4)

print(f"전처리 결과 저장: {output_file}")